In [2]:
import pandas as pd

dataset = pd.read_csv('housing.csv')

In [3]:
print(dataset.shape)
dataset[:5]

(506, 13)


,crim,zn,indus,chas,nox,age,rm,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,5.33,36.2


In [4]:
# Move 'medv' column to front
dataset = pd.concat([dataset['medv'], dataset.drop(['medv'], axis=1)], axis=1)

In [5]:
from sklearn.model_selection import train_test_split
training_dataset, validation_dataset = train_test_split(dataset, test_size=0.1)

print(training_dataset.shape)
print(validation_dataset.shape)

(455, 13)
(51, 13)


In [6]:
training_dataset.to_csv('training_dataset.csv', index=False, header=False)
validation_dataset.to_csv('validation_dataset.csv', index=False, header=False)

In [7]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()

prefix = 'boston-housing'
training_data_path = sess.upload_data(path='training_dataset.csv', key_prefix=prefix + '/input/training')
validation_data_path = sess.upload_data(path='validation_dataset.csv', key_prefix=prefix + '/input/validation')

print(training_data_path)
print(validation_data_path)

2.88.1
s3://sagemaker-us-east-1-607098578469/boston-housing/input/training/training_dataset.csv
s3://sagemaker-us-east-1-607098578469/boston-housing/input/validation/validation_dataset.csv


In [8]:
import boto3
from sagemaker import image_uris

region = boto3.Session().region_name    
container = image_uris.retrieve('xgboost', region, version='latest')

from sagemaker.estimator import Estimator

role = sagemaker.get_execution_role() 

xgb_estimator = Estimator(container,
    role=role, 
    instance_count=1,
    instance_type='ml.m5.large',
    output_path='s3://{}/{}/output'.format(bucket, prefix)
)

xgb_estimator.set_hyperparameters(objective='reg:linear',
                                 num_round=200,
                                 early_stopping_rounds=10)

In [9]:
training_data_channel   = sagemaker.TrainingInput(s3_data=training_data_path, content_type='text/csv')
validation_data_channel = sagemaker.TrainingInput(s3_data=validation_data_path, content_type='text/csv')

xgb_data = {'train': training_data_channel, 'validation': validation_data_channel}

In [10]:
xgb_estimator.fit(xgb_data)

2022-05-18 10:20:12 Starting - Starting the training job...
2022-05-18 10:20:37 Starting - Preparing the instances for trainingProfilerReport-1652869212: InProgress
.........
2022-05-18 10:22:04 Downloading - Downloading input data......
2022-05-18 10:23:05 Training - Downloading the training image...
2022-05-18 10:23:41 Uploading - Uploading generated training modelArguments: train
[2022-05-18:10:23:34:INFO] Running standalone xgboost training.
[2022-05-18:10:23:34:INFO] File size need to be processed in the node: 0.04mb. Available memory size in the node: 265.66mb
[2022-05-18:10:23:34:INFO] Determined delimiter of CSV input is ','
[10:23:34] S3DistributionType set as FullyReplicated
[10:23:34] 455x12 matrix with 5460 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2022-05-18:10:23:34:INFO] Determined delimiter of CSV input is ','
[10:23:34] S3DistributionType set as FullyReplicated
[10:23:34] 51x12 matrix with 612 entries loaded from /opt/ml/input/

In [11]:
from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())

endpoint_name = 'xgb-demo-'+timestamp
print(endpoint_name)

xgb-demo-18-10-24-46


In [12]:
xgb_predictor = xgb_estimator.deploy(endpoint_name=endpoint_name, 
                        initial_instance_count=1, 
                        instance_type='ml.t2.medium')

------------!

In [13]:
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98'

In [14]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()
xgb_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

response = xgb_predictor.predict(test_sample)
print(response)

[['25.481184005737305']]


In [34]:
runtime = boto3.Session().client(service_name='runtime.sagemaker') 

response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType='text/csv', 
                                  Body=test_sample)

print(response['Body'].read())

b'25.481184005737305'


In [ ]:
xgb_predictor.delete_endpoint()